In [3]:
import pandas as pd
from datetime import datetime
import seaborn as sns
import swifter
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

//anaconda3/envs/learn-env/lib/python3.6/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}
//anaconda3/envs/learn-env/lib/python3.6/site-packages/swifter/swifter.py:14: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import requested from: 'numba.errors', please update to use 'numba.core.errors' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.errors import TypingError
//anaconda3/envs/learn-env/lib/python3.6/site-packages/swifter/swifter.py:14: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import of 'TypingError' requested from: 'numba.errors', please update to use 'numba.core.errors' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  fr

# Import Dataset

In [4]:
df = pd.read_csv('train.csv')

In [6]:
df.head()

,user_id,session_id,timestamp,step,action_type,reference,platform,city,device,current_filters,impressions,prices
0,00RL8Z82B2Z1,aff3928535f48,1541037460,1,search for poi,Newtown,AU,"Sydney, Australia",mobile,NaN,NaN,NaN
1,00RL8Z82B2Z1,aff3928535f48,1541037522,2,interaction item image,666856,AU,"Sydney, Australia",mobile,NaN,NaN,NaN
2,00RL8Z82B2Z1,aff3928535f48,1541037522,3,interaction item image,666856,AU,"Sydney, Australia",mobile,NaN,NaN,NaN
3,00RL8Z82B2Z1,aff3928535f48,1541037532,4,interaction item image,666856,AU,"Sydney, Australia",mobile,NaN,NaN,NaN
4,00RL8Z82B2Z1,aff3928535f48,1541037532,5,interaction item image,109038,AU,"Sydney, Australia",mobile,NaN,NaN,NaN


In [7]:
df.shape

(15932992, 12)

### Get only clickout items
 - Clickout items are where users are making a decision. Other action types have been eliminated because the goal is to find which combinations of features are the most important to users when making a final clickout. 

In [8]:
df = df[df['action_type'] == 'clickout item'].drop_duplicates(subset = ['session_id'], keep = 'first')

In [9]:
df.head()

,user_id,session_id,timestamp,step,action_type,reference,platform,city,device,current_filters,impressions,prices
13,00RL8Z82B2Z1,aff3928535f48,1541037543,14,clickout item,109038,AU,"Sydney, Australia",mobile,NaN,3400638|1253714|3367857|5100540|1088584|666916...,95|66|501|112|95|100|101|72|82|56|56|143|70|25...
115,02SRUT1NQYH1,3599a6f709eab,1541063864,35,clickout item,2795374,FI,"Krakow, Poland",mobile,NaN,2795374|5582964|1088390|2781070|1258068|127196...,64|54|36|121|76|81|92|40|73|52|98|104|56|414|6...
121,03K8AXBL4BX2,ec139e10b9238,1541100322,6,clickout item,1032816,UK,"London, United Kingdom",desktop,NaN,12693|46363|81657|18448|47687|152913|18417|927...,104|92|100|103|102|104|72|85|81|75|107|86|98|8...
176,03P4VFKK12UO,325fafb5fa450,1541107500,54,clickout item,65685,US,"Bakersfield, USA",desktop,Very Good Rating|5 Star|4 Star|Hotel|Motel|Res...,1306936|56482|2842358|6881276|65685|63259|6539...,178|104|110|94|57|96|46|61|48|35|50|38|59|44|5...
180,0473FZ8UNXRS,bcc452f3350eb,1541062532,3,clickout item,3143258,AU,"Legian, Indonesia",desktop,NaN,1258184|3866722|8929970|2315702|116619|1511641...,51|43|69|49|62|50|55|42|87|46|43|114|194|50|19...


## Determine which cities are them most popular 

In [10]:
df.city.value_counts()

London, United Kingdom    12625
Tokyo, Japan               9475
New York, USA              8514
Paris, France              8381
Las Vegas, USA             7255
                          ...  
Heping, China                 1
Ostheim, France               1
Rosières, France              1
Torbalı, Turkey               1
St. Paul, USA                 1
Name: city, Length: 29625, dtype: int64

## We will use the 3 most popular cities for initial evaluation
 1. London
 2. Tokyo
 3. New York 

# London 

In [11]:
df.head()

,user_id,session_id,timestamp,step,action_type,reference,platform,city,device,current_filters,impressions,prices
13,00RL8Z82B2Z1,aff3928535f48,1541037543,14,clickout item,109038,AU,"Sydney, Australia",mobile,NaN,3400638|1253714|3367857|5100540|1088584|666916...,95|66|501|112|95|100|101|72|82|56|56|143|70|25...
115,02SRUT1NQYH1,3599a6f709eab,1541063864,35,clickout item,2795374,FI,"Krakow, Poland",mobile,NaN,2795374|5582964|1088390|2781070|1258068|127196...,64|54|36|121|76|81|92|40|73|52|98|104|56|414|6...
121,03K8AXBL4BX2,ec139e10b9238,1541100322,6,clickout item,1032816,UK,"London, United Kingdom",desktop,NaN,12693|46363|81657|18448|47687|152913|18417|927...,104|92|100|103|102|104|72|85|81|75|107|86|98|8...
176,03P4VFKK12UO,325fafb5fa450,1541107500,54,clickout item,65685,US,"Bakersfield, USA",desktop,Very Good Rating|5 Star|4 Star|Hotel|Motel|Res...,1306936|56482|2842358|6881276|65685|63259|6539...,178|104|110|94|57|96|46|61|48|35|50|38|59|44|5...
180,0473FZ8UNXRS,bcc452f3350eb,1541062532,3,clickout item,3143258,AU,"Legian, Indonesia",desktop,NaN,1258184|3866722|8929970|2315702|116619|1511641...,51|43|69|49|62|50|55|42|87|46|43|114|194|50|19...


## Sort Data by City 

In [12]:
def sort_by_city(df, city):
    city_df = df[df['city'] == city]
    return city_df
    

In [13]:
london_df = sort_by_city(df, 'London, United Kingdom')

In [14]:
london_df.head()

,user_id,session_id,timestamp,step,action_type,reference,platform,city,device,current_filters,impressions,prices
121,03K8AXBL4BX2,ec139e10b9238,1541100322,6,clickout item,1032816,UK,"London, United Kingdom",desktop,NaN,12693|46363|81657|18448|47687|152913|18417|927...,104|92|100|103|102|104|72|85|81|75|107|86|98|8...
4950,AL1GXKFSX17S,b8298d3a5830d,1541108234,27,clickout item,12695,AT,"London, United Kingdom",desktop,NaN,1819741|46317|47115|1670405|12695|47672|46574|...,214|186|208|259|213|203|130|156|195|222|234|21...
5926,DKEJ7CIFCAJ9,069e185e3c8b3,1541080069,6,clickout item,13845,HK,"London, United Kingdom",mobile,NaN,13845|5722432|149078|1670405|12710|12763|13968...,105|115|171|166|127|152|590|132|85|92|43|71|13...
5954,DPC398J0NG7A,9d6bd4ac6386b,1541033963,13,clickout item,2195678,FR,"London, United Kingdom",mobile,NaN,2195678|12681|46468|926625|46535|1160364|93817...,235|282|377|309|913|2067|242|601|255|259|295|3...
6432,FBRAKWKY8KK6,2d2c12c926cf3,1541085283,12,clickout item,664366,UK,"London, United Kingdom",mobile,NaN,664366|6728838|3366712|920457|18456|2097720|26...,112|74|92|118|163|192|67|144|139|93|310|146|84...


In [15]:
london_df.shape

(12625, 12)

### Remove pipes and place values in a list

In [16]:
def get_impressions(doc):
    doc = str(doc)
    #doc = re.sub(r'(\|)', ' ', doc)
    doc = doc.split('|')
    return doc

### Applying the get_impressions to the dataframe

In [17]:
def apply_get(df, newcol, col):
    df[newcol] = df[col].swifter.apply(get_impressions)
    df.drop(col, axis=1, inplace=True)
    return df

In [18]:
apply_get(london_df, 'price', 'prices')

//anaconda3/envs/learn-env/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
//anaconda3/envs/learn-env/lib/python3.6/site-packages/pandas/core/frame.py:4102: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,user_id,session_id,timestamp,step,action_type,reference,platform,city,device,current_filters,impressions,price
121,03K8AXBL4BX2,ec139e10b9238,1541100322,6,clickout item,1032816,UK,"London, United Kingdom",desktop,NaN,12693|46363|81657|18448|47687|152913|18417|927...,"[104, 92, 100, 103, 102, 104, 72, 85, 81, 75, ..."
4950,AL1GXKFSX17S,b8298d3a5830d,1541108234,27,clickout item,12695,AT,"London, United Kingdom",desktop,NaN,1819741|46317|47115|1670405|12695|47672|46574|...,"[214, 186, 208, 259, 213, 203, 130, 156, 195, ..."
5926,DKEJ7CIFCAJ9,069e185e3c8b3,1541080069,6,clickout item,13845,HK,"London, United Kingdom",mobile,NaN,13845|5722432|149078|1670405|12710|12763|13968...,"[105, 115, 171, 166, 127, 152, 590, 132, 85, 9..."
5954,DPC398J0NG7A,9d6bd4ac6386b,1541033963,13,clickout item,2195678,FR,"London, United Kingdom",mobile,NaN,2195678|12681|46468|926625|46535|1160364|93817...,"[235, 282, 377, 309, 913, 2067, 242, 601, 255,..."
6432,FBRAKWKY8KK6,2d2c12c926cf3,1541085283,12,clickout item,664366,UK,"London, United Kingdom",mobile,NaN,664366|6728838|3366712|920457|18456|2097720|26...,"[112, 74, 92, 118, 163, 192, 67, 144, 139, 93,..."
...,...,...,...,...,...,...,...,...,...,...,...,...
15928395,LVTHS2MARKGO,ed08f69e481a5,1541515959,1,clickout item,98752,IT,"London, United Kingdom",mobile,NaN,46571|81664|1412769|98752|47155|48099|1073868|...,"[844, 235, 532, 249, 343, 228, 144, 250, 371, ..."
15929965,R0H3JB8AO40L,f97fd1cd3f9da,1541536223,1,clickout item,3755574,UK,"London, United Kingdom",mobile,NaN,3755574|6696124|5833050|7809234|46884|18439|12...,"[396, 585, 117, 420, 406, 441, 494, 526, 347, ..."
15930734,SW9S5F3MILDI,2ccc70c40055f,1541535137,2,clickout item,6437326,US,"London, United Kingdom",mobile,NaN,894693|46408|81700|47433|81671|152883|46930|94...,"[104, 74, 92, 107, 57, 136, 151, 99, 60, 45, 6..."
15930746,SXGJ96Z0I8D0,c89fc3eaf38fa,1541537924,12,clickout item,48161,UK,"London, United Kingdom",tablet,Sort by Price|5 Star,2627696|46547|48071|2489180|47075|47136|47717|...,"[413, 451, 494, 503, 517, 656, 678, 725, 755, ..."


In [19]:
apply_get(london_df, 'impression', 'impressions')

//anaconda3/envs/learn-env/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,user_id,session_id,timestamp,step,action_type,reference,platform,city,device,current_filters,price,impression
121,03K8AXBL4BX2,ec139e10b9238,1541100322,6,clickout item,1032816,UK,"London, United Kingdom",desktop,NaN,"[104, 92, 100, 103, 102, 104, 72, 85, 81, 75, ...","[12693, 46363, 81657, 18448, 47687, 152913, 18..."
4950,AL1GXKFSX17S,b8298d3a5830d,1541108234,27,clickout item,12695,AT,"London, United Kingdom",desktop,NaN,"[214, 186, 208, 259, 213, 203, 130, 156, 195, ...","[1819741, 46317, 47115, 1670405, 12695, 47672,..."
5926,DKEJ7CIFCAJ9,069e185e3c8b3,1541080069,6,clickout item,13845,HK,"London, United Kingdom",mobile,NaN,"[105, 115, 171, 166, 127, 152, 590, 132, 85, 9...","[13845, 5722432, 149078, 1670405, 12710, 12763..."
5954,DPC398J0NG7A,9d6bd4ac6386b,1541033963,13,clickout item,2195678,FR,"London, United Kingdom",mobile,NaN,"[235, 282, 377, 309, 913, 2067, 242, 601, 255,...","[2195678, 12681, 46468, 926625, 46535, 1160364..."
6432,FBRAKWKY8KK6,2d2c12c926cf3,1541085283,12,clickout item,664366,UK,"London, United Kingdom",mobile,NaN,"[112, 74, 92, 118, 163, 192, 67, 144, 139, 93,...","[664366, 6728838, 3366712, 920457, 18456, 2097..."
...,...,...,...,...,...,...,...,...,...,...,...,...
15928395,LVTHS2MARKGO,ed08f69e481a5,1541515959,1,clickout item,98752,IT,"London, United Kingdom",mobile,NaN,"[844, 235, 532, 249, 343, 228, 144, 250, 371, ...","[46571, 81664, 1412769, 98752, 47155, 48099, 1..."
15929965,R0H3JB8AO40L,f97fd1cd3f9da,1541536223,1,clickout item,3755574,UK,"London, United Kingdom",mobile,NaN,"[396, 585, 117, 420, 406, 441, 494, 526, 347, ...","[3755574, 6696124, 5833050, 7809234, 46884, 18..."
15930734,SW9S5F3MILDI,2ccc70c40055f,1541535137,2,clickout item,6437326,US,"London, United Kingdom",mobile,NaN,"[104, 74, 92, 107, 57, 136, 151, 99, 60, 45, 6...","[894693, 46408, 81700, 47433, 81671, 152883, 4..."
15930746,SXGJ96Z0I8D0,c89fc3eaf38fa,1541537924,12,clickout item,48161,UK,"London, United Kingdom",tablet,Sort by Price|5 Star,"[413, 451, 494, 503, 517, 656, 678, 725, 755, ...","[2627696, 46547, 48071, 2489180, 47075, 47136,..."


### Explode the dataframe to contain price exploded 

In [20]:
def explode_df(df, col):
    df = df.explode(col)
    return df

In [21]:
price_london_df = explode_df(london_df, 'price')

In [22]:
price_london_df.head()

,user_id,session_id,timestamp,step,action_type,reference,platform,city,device,current_filters,price,impression
121,03K8AXBL4BX2,ec139e10b9238,1541100322,6,clickout item,1032816,UK,"London, United Kingdom",desktop,NaN,104,"[12693, 46363, 81657, 18448, 47687, 152913, 18..."
121,03K8AXBL4BX2,ec139e10b9238,1541100322,6,clickout item,1032816,UK,"London, United Kingdom",desktop,NaN,92,"[12693, 46363, 81657, 18448, 47687, 152913, 18..."
121,03K8AXBL4BX2,ec139e10b9238,1541100322,6,clickout item,1032816,UK,"London, United Kingdom",desktop,NaN,100,"[12693, 46363, 81657, 18448, 47687, 152913, 18..."
121,03K8AXBL4BX2,ec139e10b9238,1541100322,6,clickout item,1032816,UK,"London, United Kingdom",desktop,NaN,103,"[12693, 46363, 81657, 18448, 47687, 152913, 18..."
121,03K8AXBL4BX2,ec139e10b9238,1541100322,6,clickout item,1032816,UK,"London, United Kingdom",desktop,NaN,102,"[12693, 46363, 81657, 18448, 47687, 152913, 18..."


In [23]:
price_london_df.shape

(310967, 12)

### Reset index to avoid error of duplicate index


In [24]:
price_london_df.reset_index()

,index,user_id,session_id,timestamp,step,action_type,reference,platform,city,device,current_filters,price,impression
0,121,03K8AXBL4BX2,ec139e10b9238,1541100322,6,clickout item,1032816,UK,"London, United Kingdom",desktop,NaN,104,"[12693, 46363, 81657, 18448, 47687, 152913, 18..."
1,121,03K8AXBL4BX2,ec139e10b9238,1541100322,6,clickout item,1032816,UK,"London, United Kingdom",desktop,NaN,92,"[12693, 46363, 81657, 18448, 47687, 152913, 18..."
2,121,03K8AXBL4BX2,ec139e10b9238,1541100322,6,clickout item,1032816,UK,"London, United Kingdom",desktop,NaN,100,"[12693, 46363, 81657, 18448, 47687, 152913, 18..."
3,121,03K8AXBL4BX2,ec139e10b9238,1541100322,6,clickout item,1032816,UK,"London, United Kingdom",desktop,NaN,103,"[12693, 46363, 81657, 18448, 47687, 152913, 18..."
4,121,03K8AXBL4BX2,ec139e10b9238,1541100322,6,clickout item,1032816,UK,"London, United Kingdom",desktop,NaN,102,"[12693, 46363, 81657, 18448, 47687, 152913, 18..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
310962,15932569,YH5BH5INZMHX,068dcaa3b08b4,1541518492,1,clickout item,1160332,UK,"London, United Kingdom",desktop,NaN,447,"[12669, 1160332, 47617, 6255610, 46875, 47412,..."
310963,15932569,YH5BH5INZMHX,068dcaa3b08b4,1541518492,1,clickout item,1160332,UK,"London, United Kingdom",desktop,NaN,366,"[12669, 1160332, 47617, 6255610, 46875, 47412,..."
310964,15932569,YH5BH5INZMHX,068dcaa3b08b4,1541518492,1,clickout item,1160332,UK,"London, United Kingdom",desktop,NaN,531,"[12669, 1160332, 47617, 6255610, 46875, 47412,..."
310965,15932569,YH5BH5INZMHX,068dcaa3b08b4,1541518492,1,clickout item,1160332,UK,"London, United Kingdom",desktop,NaN,90,"[12669, 1160332, 47617, 6255610, 46875, 47412,..."


### Creating a new dataframe with the same process, for impressions, then replacing the column in country df with the new values 

In [25]:
imp_london_df = explode_df(london_df, 'impression')

In [26]:
imp_london_df.reset_index()

,index,user_id,session_id,timestamp,step,action_type,reference,platform,city,device,current_filters,price,impression
0,121,03K8AXBL4BX2,ec139e10b9238,1541100322,6,clickout item,1032816,UK,"London, United Kingdom",desktop,NaN,"[104, 92, 100, 103, 102, 104, 72, 85, 81, 75, ...",12693
1,121,03K8AXBL4BX2,ec139e10b9238,1541100322,6,clickout item,1032816,UK,"London, United Kingdom",desktop,NaN,"[104, 92, 100, 103, 102, 104, 72, 85, 81, 75, ...",46363
2,121,03K8AXBL4BX2,ec139e10b9238,1541100322,6,clickout item,1032816,UK,"London, United Kingdom",desktop,NaN,"[104, 92, 100, 103, 102, 104, 72, 85, 81, 75, ...",81657
3,121,03K8AXBL4BX2,ec139e10b9238,1541100322,6,clickout item,1032816,UK,"London, United Kingdom",desktop,NaN,"[104, 92, 100, 103, 102, 104, 72, 85, 81, 75, ...",18448
4,121,03K8AXBL4BX2,ec139e10b9238,1541100322,6,clickout item,1032816,UK,"London, United Kingdom",desktop,NaN,"[104, 92, 100, 103, 102, 104, 72, 85, 81, 75, ...",47687
...,...,...,...,...,...,...,...,...,...,...,...,...,...
310962,15932569,YH5BH5INZMHX,068dcaa3b08b4,1541518492,1,clickout item,1160332,UK,"London, United Kingdom",desktop,NaN,"[51, 314, 147, 143, 280, 102, 701, 229, 81, 62...",47074
310963,15932569,YH5BH5INZMHX,068dcaa3b08b4,1541518492,1,clickout item,1160332,UK,"London, United Kingdom",desktop,NaN,"[51, 314, 147, 143, 280, 102, 701, 229, 81, 62...",6475130
310964,15932569,YH5BH5INZMHX,068dcaa3b08b4,1541518492,1,clickout item,1160332,UK,"London, United Kingdom",desktop,NaN,"[51, 314, 147, 143, 280, 102, 701, 229, 81, 62...",3218556
310965,15932569,YH5BH5INZMHX,068dcaa3b08b4,1541518492,1,clickout item,1160332,UK,"London, United Kingdom",desktop,NaN,"[51, 314, 147, 143, 280, 102, 701, 229, 81, 62...",1255377


### Combining price dataframe with impressions dataframe to get the dataframe for London

In [27]:
price_london_df['impression'] = imp_london_df['impression']

In [28]:
london_df = price_london_df

In [29]:
london_df.to_csv('london_df.csv')

In [37]:
london_df.head()

,user_id,session_id,timestamp,step,action_type,reference,platform,city,device,current_filters,price,impression
121,03K8AXBL4BX2,ec139e10b9238,1541100322,6,clickout item,1032816,UK,"London, United Kingdom",desktop,NaN,104,12693
121,03K8AXBL4BX2,ec139e10b9238,1541100322,6,clickout item,1032816,UK,"London, United Kingdom",desktop,NaN,92,46363
121,03K8AXBL4BX2,ec139e10b9238,1541100322,6,clickout item,1032816,UK,"London, United Kingdom",desktop,NaN,100,81657
121,03K8AXBL4BX2,ec139e10b9238,1541100322,6,clickout item,1032816,UK,"London, United Kingdom",desktop,NaN,103,18448
121,03K8AXBL4BX2,ec139e10b9238,1541100322,6,clickout item,1032816,UK,"London, United Kingdom",desktop,NaN,102,47687


### Finding where reference matches impression to determine which combination of impressions was chosen by the user

In [39]:
def find_matches(df):
    df['match'] = np.where(df['reference'] == df['impression'], '1', '0')
    return df

In [40]:
find_matches(london_df)

,user_id,session_id,timestamp,step,action_type,reference,platform,city,device,current_filters,price,impression,match
121,03K8AXBL4BX2,ec139e10b9238,1541100322,6,clickout item,1032816,UK,"London, United Kingdom",desktop,NaN,104,12693,0
121,03K8AXBL4BX2,ec139e10b9238,1541100322,6,clickout item,1032816,UK,"London, United Kingdom",desktop,NaN,92,46363,0
121,03K8AXBL4BX2,ec139e10b9238,1541100322,6,clickout item,1032816,UK,"London, United Kingdom",desktop,NaN,100,81657,0
121,03K8AXBL4BX2,ec139e10b9238,1541100322,6,clickout item,1032816,UK,"London, United Kingdom",desktop,NaN,103,18448,0
121,03K8AXBL4BX2,ec139e10b9238,1541100322,6,clickout item,1032816,UK,"London, United Kingdom",desktop,NaN,102,47687,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
15932569,YH5BH5INZMHX,068dcaa3b08b4,1541518492,1,clickout item,1160332,UK,"London, United Kingdom",desktop,NaN,447,47074,0
15932569,YH5BH5INZMHX,068dcaa3b08b4,1541518492,1,clickout item,1160332,UK,"London, United Kingdom",desktop,NaN,366,6475130,0
15932569,YH5BH5INZMHX,068dcaa3b08b4,1541518492,1,clickout item,1160332,UK,"London, United Kingdom",desktop,NaN,531,3218556,0
15932569,YH5BH5INZMHX,068dcaa3b08b4,1541518492,1,clickout item,1160332,UK,"London, United Kingdom",desktop,NaN,90,1255377,0


In [44]:
london_df.match.value_counts()

0    298347
1     12620
Name: match, dtype: int64

### Creating a new dataframe to show the number of impressions for each session

In [45]:
def create_ref_count_df(df):
    count_df = df.groupby(['session_id'])['impression'].count()
    pd.DataFrame(count_df)
    return count_df

### Create the new series and convert it to dataframe

In [46]:
ref_count_df = pd.DataFrame(create_ref_count_df(london_df))

### Drop rows with less than 25 impressions to maintain model accuracy

In [47]:
acc_df = ref_count_df.drop( ref_count_df[ ref_count_df['impression'] < 25 ].index)

In [48]:
acc_df

,impression
session_id,
0005fe7961566,25
00079f55efc6f,25
0008e0d108065,25
000b186d74d7d,25
000e2e48d7658,25
...,...
ffdc7c1aeaeb3,25
ffdf378d8ef0b,25
fff4202576d34,25


In [49]:
acc_df.impression.nunique()

1

### Merge the accurate dataframe with country dataframe

In [50]:
new_df = pd.merge(london_df, acc_df, on='session_id', how='outer')

In [52]:
new_df.head()

,user_id,session_id,timestamp,step,action_type,reference,platform,city,device,current_filters,price,impression_x,match,impression_y
0,03K8AXBL4BX2,ec139e10b9238,1541100322,6,clickout item,1032816,UK,"London, United Kingdom",desktop,NaN,104,12693,0,25.0
1,03K8AXBL4BX2,ec139e10b9238,1541100322,6,clickout item,1032816,UK,"London, United Kingdom",desktop,NaN,92,46363,0,25.0
2,03K8AXBL4BX2,ec139e10b9238,1541100322,6,clickout item,1032816,UK,"London, United Kingdom",desktop,NaN,100,81657,0,25.0
3,03K8AXBL4BX2,ec139e10b9238,1541100322,6,clickout item,1032816,UK,"London, United Kingdom",desktop,NaN,103,18448,0,25.0
4,03K8AXBL4BX2,ec139e10b9238,1541100322,6,clickout item,1032816,UK,"London, United Kingdom",desktop,NaN,102,47687,0,25.0


### Creating a column to identify how many steps a user took 

In [53]:
new_df['count'] = new_df.groupby((new_df['session_id'] != new_df['session_id'].shift(1)).cumsum()).cumcount()+1

In [62]:
london_df = new_df

### The chosen impression is impression_x, this will be renamed impression to avoid confusion

In [63]:
london_df.rename({'impression_x':'impression'},axis=1, inplace = True)

In [64]:
london_df.head()

,user_id,session_id,timestamp,step,action_type,reference,platform,city,device,current_filters,price,impression,match,impression_y,count
0,03K8AXBL4BX2,ec139e10b9238,1541100322,6,clickout item,1032816,UK,"London, United Kingdom",desktop,NaN,104,12693,0,25.0,1
1,03K8AXBL4BX2,ec139e10b9238,1541100322,6,clickout item,1032816,UK,"London, United Kingdom",desktop,NaN,92,46363,0,25.0,2
2,03K8AXBL4BX2,ec139e10b9238,1541100322,6,clickout item,1032816,UK,"London, United Kingdom",desktop,NaN,100,81657,0,25.0,3
3,03K8AXBL4BX2,ec139e10b9238,1541100322,6,clickout item,1032816,UK,"London, United Kingdom",desktop,NaN,103,18448,0,25.0,4
4,03K8AXBL4BX2,ec139e10b9238,1541100322,6,clickout item,1032816,UK,"London, United Kingdom",desktop,NaN,102,47687,0,25.0,5


# Modeling

In [65]:
london_model_df = london_df[['price', 'match', 'count', 'impression']]

In [66]:
london_model_df

,price,match,count,impression
0,104,0,1,12693
1,92,0,2,46363
2,100,0,3,81657
3,103,0,4,18448
4,102,0,5,47687
...,...,...,...,...
310962,447,0,21,47074
310963,366,0,22,6475130
310964,531,0,23,3218556
310965,90,0,24,1255377


In [67]:
ohe_london = pd.read_csv('ohe_lon.csv')

FileNotFoundError: [Errno 2] File b'ohe_lon.csv' does not exist: b'ohe_lon.csv'